# Desafío 2 - PNL

#### 1 -  Crear sus propios vectores con Gensim basado en lo visto en clase con otro dataset.
#### 2 - Probar términos de interés y explicar similitudes en el espacio de embeddings.
#### 3 - Intentar plantear y probar tests de analogías.
####4 - Graficar los embeddings resultantes. Sacar conclusiones.

# Procesamiento de lenguaje natural
## Custom embedddings con Gensim



### Objetivo
El objetivo planteado es utilizar un libro de la historia de la Europa, donde vamos analizar los embeddings:

#### Empezamos por instalar las librerias necesarias

In [ ]:
!pip install tensorflow

In [ ]:
import os
import platform
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import multiprocessing
from gensim.models import Word2Vec

### Datos

Cargando el texto de la historia de la Europa:
https://www.kaggle.com/datasets/chungimungi/europehistorytxt?resource=download

In [ ]:
# Armar el dataset utilizando salto de línea para separar las oraciones/docs
df = pd.read_csv('desafio_2_europe_history.txt', sep='/n', header=None)
df.head()

<ipython-input-78-241da0643894>:2: ParserWarning:

Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.



,0
0,PREFACE
1,This book contains little that is original. Si...
2,been thoroughly worked over by previous histor...
3,"rarely required. The book's originality, such ..."
4,"rearrangement, and presentation of the content..."


In [ ]:
print("Cantidad de documentos:", df.shape[0])

Cantidad de documentos: 48008


### 1 - Preprocesamiento

In [ ]:
from tensorflow.keras.preprocessing.text import text_to_word_sequence

sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también)
for _, row in df[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row[0]))

In [ ]:
# Demos un vistazo
sentence_tokens[:2]

[['preface'],
 ['this',
  'book',
  'contains',
  'little',
  'that',
  'is',
  'original',
  'since',
  'most',
  'aspects',
  'of',
  'the',
  'subject',
  'have']]

# 1 -  Crear sus propios vectores con Gensim basado en lo visto en clase con otro dataset.

### Creando los vectores (word2vec)

In [ ]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobrecargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [ ]:
w2v_model = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=10,       # cant de palabras antes y desp de la predicha
                     vector_size=250,       # dimensionalidad de los vectores - BAJAMOS
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=multiprocessing.cpu_count(),      # si tienen más cores pueden cambiar este valor
                     sg=0)           # modelo 0:CBOW  1:skipgram

In [ ]:
# Obtener el vocabulario con los tokens
w2v_model.build_vocab(sentence_tokens)

In [ ]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 48008


In [ ]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.index_to_key))

Cantidad de words distintas en el corpus: 10341


### Entrenando embeddings

In [ ]:
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=300,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 414404.5625
Loss after epoch 1: 414554.375
Loss after epoch 2: 404567.5625
Loss after epoch 3: 394123.375
Loss after epoch 4: 389854.375
Loss after epoch 5: 376756.25
Loss after epoch 6: 360389.75
Loss after epoch 7: 369354.0
Loss after epoch 8: 367720.75
Loss after epoch 9: 367815.25
Loss after epoch 10: 360333.25
Loss after epoch 11: 343480.0
Loss after epoch 12: 342430.0
Loss after epoch 13: 329569.5
Loss after epoch 14: 338179.0
Loss after epoch 15: 334310.0
Loss after epoch 16: 337104.5
Loss after epoch 17: 336831.5
Loss after epoch 18: 335808.5
Loss after epoch 19: 333423.0
Loss after epoch 20: 335804.0
Loss after epoch 21: 334270.0
Loss after epoch 22: 334376.0
Loss after epoch 23: 323709.5
Loss after epoch 24: 324130.0
Loss after epoch 25: 319314.0
Loss after epoch 26: 319854.0
Loss after epoch 27: 320083.0
Loss after epoch 28: 316636.0
Loss after epoch 29: 318489.0
Loss after epoch 30: 330406.0
Loss after epoch 31: 318683.0
Loss after epoch 32: 317077.0
Los

(109238878, 165243600)

# 2 - Probar términos de interés y explicar similitudes en el espacio de embeddings.

### Ensayar

In [ ]:
# prompt: explicar similitudes en el espacio de embeddings y hacer una conclusion

# 2 - Probar términos de interés y explicar similitudes en el espacio de embeddings.
# ### Ensayar
terms = ['europe', 'war', 'politics', 'france', 'germany', 'england', 'day', 'napoleon', 'revolution']
for term in terms:
    if term in w2v_model.wv:
        print(f"\nPalabras similares a '{term}':")
        for similar_word, similarity in w2v_model.wv.most_similar(term, topn=5):  # Mostrar las 5 más similares
            print(f"- {similar_word}: {similarity:.4f}")
    else:
        print(f"\nLa palabra '{term}' no se encuentra en el vocabulario del modelo.")


Palabras similares a 'europe':
- europe's: 0.3372
- sector: 0.2916
- the: 0.2515
- it: 0.2480
- european: 0.2458

Palabras similares a 'war':
- wars: 0.3055
- charles's: 0.2477
- urgency: 0.2461
- louvre: 0.2445
- campaign: 0.2393

Palabras similares a 'politics':
- manifestly: 0.2374
- deutschmark: 0.2177
- manifestations: 0.2164
- steam: 0.2154
- block: 0.2135

Palabras similares a 'france':
- sweden: 0.2634
- ireland: 0.2404
- shortages: 0.2274
- savoy: 0.2223
- triple: 0.2166

Palabras similares a 'germany':
- europe: 0.2454
- front: 0.2199
- russia: 0.2097
- republicanism: 0.2091
- britain: 0.2070

Palabras similares a 'england':
- britain: 0.2832
- toulouse: 0.2406
- norwegians: 0.2277
- livonia: 0.2238
- 1643: 0.2225

Palabras similares a 'day':
- evening: 0.2795
- occasion: 0.2758
- week: 0.2647
- month: 0.2505
- night: 0.2462

Palabras similares a 'napoleon':
- temper: 0.2356
- horse: 0.2120
- ubiquitous: 0.2062
- survivors: 0.2049
- deserved: 0.2013

Palabras similares a 're

In [ ]:
# prompt: Implementar y probar tests de analogías. Ejemplo de analogía: "Francia" es a "París" lo que "Alemania" es a "Berlín"

# Verificar si las palabras están en el vocabulario
words_to_check = ['france', 'paris', 'germany', 'berlin']
for word in words_to_check:
    if word not in w2v_model.wv:
        print(f"La palabra '{word}' no está en el vocabulario. No se puede realizar la analogía.")
        exit()  # O manejar el error de otra manera

# Calcular la analogía
try:
    result = w2v_model.wv.most_similar(positive=['germany', 'paris'], negative=['france'], topn=1)
    print(f"La analogía de 'Francia' es a 'París' lo que 'Alemania' es a: {result[0][0]} con una similitud de: {result[0][1]:.4f}")
except KeyError as e:
    print(f"Error al calcular la analogía: {e}")

# Probar otras analogías
analogias = [
    ("france", "paris", "germany", "berlin"),
    ("england", "london", "italy", "rome"), # Ejemplo con otra analogía, puede que no funcione correctamente
    ("europe", "war", "politics", "peace"), #Analogía que puede funcionar
    ("king", "man", "queen", "woman")
]

for a, b, c, expected in analogias:
    try:
        result = w2v_model.wv.most_similar(positive=[c, b], negative=[a], topn=1)
        print(f"'{a}' es a '{b}' lo que '{c}' es a: {result[0][0]} con similitud de {result[0][1]:.4f}, se esperaba: {expected}")

    except KeyError as e:
        print(f"Error al calcular la analogía: {e}")

La analogía de 'Francia' es a 'París' lo que 'Alemania' es a: cologne con una similitud de: 0.2803
'france' es a 'paris' lo que 'germany' es a: cologne con similitud de 0.2803, se esperaba: berlin
'england' es a 'london' lo que 'italy' es a: smyrna con similitud de 0.2280, se esperaba: rome
'europe' es a 'war' lo que 'politics' es a: travel con similitud de 0.2539, se esperaba: peace
'king' es a 'man' lo que 'queen' es a: devotional con similitud de 0.2532, se esperaba: woman


### 5 - Visualizar agrupación de vectores

In [ ]:
from sklearn.decomposition import IncrementalPCA
from sklearn.manifold import TSNE
import numpy as np

def reduce_dimensions(model, num_dimensions = 2 ):

    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    return vectors, labels

In [ ]:
# Graficar los embedddings en 2D
# Lleva 10 minutos para generar
import plotly.graph_objects as go
import plotly.express as px

vecs, labels = reduce_dimensions(w2v_model)

MAX_WORDS=200
fig = px.scatter(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], text=labels[:MAX_WORDS])
fig.show(renderer="colab") # esto para plotly en colab

In [ ]:
# Graficar los embedddings en 3D
# Lleva 15 minutos en procesamiento

vecs, labels = reduce_dimensions(w2v_model,3)

fig = px.scatter_3d(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], z=vecs[:MAX_WORDS,2],text=labels[:MAX_WORDS])
fig.update_traces(marker_size = 2)
fig.show(renderer="colab") # esto para plotly en colab

# Conclusiones:
##### Se observa que el modelo identifica relaciones semánticas entre las palabras. Por ejemplo, palabras relacionadas con la guerra, como 'war' o 'napoleon' aparecen juntas o cercanas en el espacio de embeddings. Se pueden observar relaciones de ubicación geográfica ('france','germany','england', 'russia') o conceptos históricos ('revolution').
##### Si bien el modelo encuentra algunas relaciones lógicas, es posible que la precisión de las similitudes no sea perfecta como se puede observar en las analogías debido a la limitación del conjunto de datos o al entrenamiento del modelo (número de epochs, parámetros del word2vec, tamaño de los vectores). El numero de epochs fue bajo debido al tiempo disponible para hacer el entrenamiento con los datos.

### En mi opinión para mejorar el modelo, se puede considerar:
##### * Un conjunto de datos más grande y ayustes de los hiperparámetros como el tamaño del vector y otros para mejorar la precisión de los vectores.
##### * Explorar/Implementar otros modelos de word embeddings como FastText o GloVe.
